#### 1. 二叉树节点类

In [1]:
# -*- coding: UTF-8 -*-
class BinNode(object):
    def __init__(self, data, parent=None, lc=None, rc=None, height=0,):
        self.data = data
        self.parent = parent
        self.height = height
        self.lc = lc
        self.rc = rc

    def __eq__(self, other):
        return self.data == other.data

    def __lt__(self, other):
        return self.data < other.data
    
    def insert_aslchild(self,data):
        new_node = BinNode(data,self)
        self.lc = new_node
        return new_node
    
    def insert_asrchild(self,data):
        new_node = BinNode(data,self)
        self.rc = new_node
        return new_node

if __name__ == "__main__":
    n = BinNode(2)
    k = BinNode(2)

#### 2. 二叉树节点工具类
将判断二叉节点状态,性质. 定位与之相关的叔叔,兄弟节点等逻辑封装

In [2]:
class BinNodeUtil(object):
    @staticmethod
    def is_root(node):
        return not node.parent
    @staticmethod
    def is_lchild(node):
        return (not BinNodeUtil.is_root(node))&(node.parent.lc is node)
    @staticmethod
    def is_rchild(node):
        return (not BinNodeUtil.is_root(node))&(node.parent.rc is node)
    @staticmethod
    def has_parent(node):
        return not BinNodeUtil.is_root(node)
    @staticmethod
    def has_lchild(node):
        return not node.lc is None
    @staticmethod
    def has_rchild(node):
        return not node.rc is None
    @staticmethod
    def has_child(node):
        return BinNodeUtil.has_lchild(node) or BinNodeUtil.has_rchild(node)
    @staticmethod
    def has_bothchild(node):
        return BinNodeUtil.has_lchild(node) & BinNodeUtil.has_rchild(node)
    @staticmethod
    def is_leaf(node):
        return not BinNodeUtil.has_child(node)
    @staticmethod
    def sibling(node):
        try:
            return  node.parent.rc if BinNodeUtil.is_lchild(node) else node.parent.lc
        except  AttributeError:
            return None
    @staticmethod
    def uncile(node):
        try:
            parent = node.parent
            return parent.parent.rc if BinNodeUtil.is_lchild(parent) else parent.parent.lc
        except  AttributeError:
            return None
    @staticmethod
    def from_parent_to(node,value=None):
        '''来自父亲的引用'''
        if (BinNodeUtil.is_lchild(node)):
            node.parent.lc = None
        if (BinNodeUtil.is_rchild(node)):
            node.parent.rc = None
    

#### 3. 二叉树类
1. 二叉树类包含二叉树的常用操作  
  1. 更新树中, 每个节点的高度  
    1. 二叉树任意节点的高度 = 其孩子节点的最大高度+1  `update_height`  
    2. 一旦有节点插入或删除, 要更新其所有祖先的高度  `update_height_above`   
  2. 插入结点到二叉树中, 3种方式  :  
    1. 插入根节点`insert_root`    
    2. 插入树中某个节点的左孩子`insert_as_lc`  
    3. 插入树中某个节点的右孩子`insert_as_rc`
  3. 子树接入  
   任意二叉树, 都可以作为另一个二叉树的左子树/右子树插入`attach_as_lc`
  4. 子树删除  
   子树删除要删除该节点和节点的所有孩子节点, 并更新树的size和height


In [3]:
class BinTree(object):
    def __init__(self):
        self.size = 0
        self.root = None
        
    def empty(self):
        return self.root is None
    
    def __eq__(self,other):
        return (self.root is not None) and (other.root is not None) and (self.root is other.root)
    
    def stature(self,node):
        return -1 if node is None else node.height
    
    def update_height(self,node):
        node.height = 1 + max(self.stature(node.lc), self.stature(node.rc))
        return node.height
    
    def update_height_above(self,node):
        while node is not None:
            self.update_height(node)
            node = node.parent
            
    def insert_root(self,data):
        self.size = 1
        self.root = BinNode(data)
        return self.root
    
    def insert_as_lc(self, node,data):
        self.size = self.size + 1
        lc = node.insert_aslchild(data)
        self.update_height_above(node)
        return lc
    
    def insert_as_rc(self, node , data):
        self.size = self.size + 1
        rc = node.insert_asrchild(data)
        self.update_height_above(node)
        return rc
    
    def attach_as_lc(self, node,other_tree):
        '''node: 该二叉树中的节点
           other_tree : 作为子树接入该二叉树. 接入点为node节点的左孩子'''
        node.lc = other_tree.root
        other_tree.root.parent = node
        self.size = self.size + other_tree.size
        self.update_height_above(node)
        return node
    
    def attach_as_rc(self, node,other_tree):
        '''node: 该二叉树中的节点
           other_tree : 作为子树接入该二叉树. 接入点为node节点的右孩子'''
        node.rc = other_tree.root
        other_tree.root.parent = node
        self.size = self.size + other_tree.size
        self.update_height_above(node)
        return node
    
    def remove(self,node):
        '''删除该二叉树中节点node及其后代'''
        n = self.removeAt(node)
        
        BinNodeUtil.from_parent_to(node,None)
        self.update_height_above(node.parent)
        self.size = self.size - n
        return n
        
    def removeAt(self,node):
        '''删除节点node及其后代'''
        if node is None:
            return 0
        n = 1 + self.removeAt(node.lc) + self.removeAt(node.rc)
        node = None 
        return n
    

#### 4. 二叉树工具类   
2. 遍历 , 树形如下 : 
  <img src='img/bintreeinR.png' width='50%' height='50%'>
   1. 递归版中序遍历`travIn_R` 
   2. 层次遍历 `travLevel`
   3. 中序遍历下, 节点的直接后继 `succ`  
    中序遍历下直接后继有两种情况  
     1. 若当前节点有右孩子  
      则其直接后继为右子树的最左节点. 因此, 可以从右孩子节点开始, 一直向左深入, 知道达到最左侧节点
     2. 若当前节点无右孩子  
      则其直接后继为将该节点作为左子树的最低祖先. 因此, 可以从该节点开始, 沿着右侧通路朝左上方不断上升, 知道不能前进时, 向右上方移动一步. 如下图所示 <img src='img/succ.png' width='30%' height='30%'>

In [4]:
class  BinTreeUtility(object):
    @staticmethod
    def travIn_R(node , visit):
        if node is None : 
            return 
        BinTreeUtility.travIn_R(node.lc , visit)
        visit (node.data)
        BinTreeUtility.travIn_R(node.rc , visit)
        
    @staticmethod
    def travLevel(node,visit):
        queue = [node]
        while len(queue)>0 :
            top_node = queue[0]  # 取队列第一个元素
            queue = queue[1:]
            visit (top_node.data)
            if top_node.lc is not None : 
                queue.append(top_node.lc)
            if top_node.rc is not None : 
                queue.append(top_node.rc)
                
    @staticmethod
    def succ(node):
        if node.rc is not None :
            node_now = node.rc
            while node_now.lc is not None:
                node_now = node_now.lc
        else : 
            node_now = node
            while BinNodeUtil.is_rchild(node_now):  # 一直向左上方提升
                node_now = node_now.parent 
            node_now = node_now.parent 
        return node_now
            

In [5]:
## 测试二叉树工具类
if __name__ == "__main__":
    # 构建树
    tree = BinTree()
    i = tree.insert_root('i')
    d =  tree.insert_as_lc(i,'d')
    l =  tree.insert_as_rc(i,'l')
    c = tree.insert_as_lc(d,'c')
    h = tree.insert_as_rc(d,'h')
    a = tree.insert_as_lc(c,'a')
    b = tree.insert_as_rc(a,'b')
    f = tree.insert_as_lc(h,'f')
    e =tree.insert_as_lc(f,'e')
    g = tree.insert_as_rc(f,'g')
    k = tree.insert_as_lc(l,'k')
    n = tree.insert_as_rc(l,'n')
    j = tree.insert_as_lc(k,'j')
    m = tree.insert_as_lc(n,'m')
    p = tree.insert_as_rc(n,'p')
    o = tree.insert_as_lc(p,'o')
   
    orders = []
    def visit(x):
        orders.append(x)
    # 遍历树
    BinTreeUtility.travLevel(tree.root,visit)
    print '层次遍历', orders
    print '中序遍历下, i 的直接后继: ', BinTreeUtility.succ(i).data

层次遍历 ['i', 'd', 'l', 'c', 'h', 'k', 'n', 'a', 'f', 'j', 'm', 'p', 'b', 'e', 'g', 'o']
中序遍历下, i 的直接后继:  j


#### 5. 二进制编码
1. 什么是二进制编码  
 将字符转变为01编码成为编码. 将所有字符都找到对应的01编码, 形成编码表. 
 <img src='img/bianmabiao.png' height='70%' width='70%'>
2. 何为解码  
 相应的, 将01编码转变为字符的过程成为解码  
3. 前缀无歧义编码 PFC  
  1. 什么时候产生歧义  
   假设字符A被编码成'11', M编码成'111', 则编码串'111111'产生歧义. 可译做"AAA"或"MM". 因为字符M的编码是字符A的编码的前缀  
  2. 反过来, 只要各字符的编码互不为前缀, 就不会产生歧义. 前缀无歧义编码也称PFC编码
4. PFC编码树  
 PFC编码要求前缀无歧义, 因此, 可采用树形结构. 从根节点出发, 每次向左/向右都对应一个0/1比特位, 从而形成编码树  
 只要所有字符都处于编码树的叶子节点, 奇异现象自然消除
 <img src='img/pfcyzjd.png' height='50%' width='50%'>
5. PFC解码  
  1. 设对编码串${110}^{01}{111}^{111}{00}^{10}{01}$解码. 只要从前向后扫面编码串, 同时在编码树中相应移动, 从根节点出发, 按照各比特位取值选择向左或向右深入一层, 直到抵达叶子节点. 从而解码出一个字符. 然后重新回到根节点, 继续解码编码串的剩余部分  
  2. 因为解码过程可以在接受编码串时实时进行, 因此PFC解码过程也称在线算法
  


#### 6. PFC编码解码编程实现
1. 因为每个PFC编码树, 都是对一个字符集的全部字符进行了编码.所以, 若2个字符集${ \Sigma  }_{ 1 }$与${ \Sigma  }_{ 2 }$没有公共字符, 则这2个字符集对应2个不同的编码树$T_1$和$T_2$. 要想得到${ \Sigma  }_{ 1 }\bigcup { { \Sigma  }_{ 2 } } $的编码树, 只要引入一个根节点, 然后合并$T_1$和$T_2$即可. 
2. PFC编码编程逻辑  
 有了上述性质, 在字符集$\Sigma$编码  
  1. 首先把每个字符构造成单节点的二叉树, 形成大小为$\left| \Sigma  \right| $的森林
  2. 反复从上面构造的森林中去除两棵树, 将其合二为一 . 经过$\left| \Sigma  \right|-1$次迭代后, 形成PFC编码树  
  3. 将PFC编码树译为编码表
3. PFC解码  
 同第5点

In [25]:
class PFCUtil(object):
    
    @classmethod
    def __initPFCForest(self,character_set):
        '''把字符集变成森林'''
        pfc_forest = []
        for character in character_set : 
            tree = BinTree()
            tree.insert_root(character)
            pfc_forest.append(tree)
        return pfc_forest
   

    @classmethod
    def pfc_tree(self,character_set):
        '''构建编码树'''
        pfc_forest = self.__initPFCForest(character_set)
        
        while len(pfc_forest)>1:
            tree1 = pfc_forest[0]
            tree2 = pfc_forest[1]
            
            merge_tree = BinTree()
            merge_tree.insert_root('^')  # 合并后的树, 根节点为'^'
            merge_tree.attach_as_lc(merge_tree.root,tree1)
            merge_tree.attach_as_rc(merge_tree.root,tree2)
            
            pfc_forest.remove(tree1)
            pfc_forest.remove(tree2)
            pfc_forest.append(merge_tree)
        code_tree = pfc_forest[0]
        return code_tree
    

    @classmethod
    def generate_code_table(self,pfc_tree):
        '''按照编码树生辰美好'''
        code_table = {}
        def gen_code(node,code):
            if (BinNodeUtil.is_leaf(node)):
                code_table[node.data] = code
                return 
            if BinNodeUtil.has_lchild(node):
                gen_code(node.lc,code+'0') 
            if BinNodeUtil.has_rchild(node):
                gen_code(node.rc,code+'1')
        root = pfc_tree.root
        gen_code(root,'')
        return code_table
        
    @classmethod
    def encode(self,sentence , pfc_tree):
        '''对字符串加密, 按照密码表'''
        code_table = self.generate_code_table(pfc_tree)
        code = ''
        for c in sentence : 
            code = code + code_table[c]
        return code
    
    @classmethod
    def decode(self,code,pfc_tree):
        '''将pfc编码按照pfc编码树解码成字符串'''
        node = pfc_tree.root
        sentence = ''
        while len(code)>0 :
            if code[0]=='0':
                node = node.lc
                code = code[1:]
            elif code[0] == '1':
                node = node.rc
                code = code[1:]
                
            if BinNodeUtil.is_leaf(node) : 
                # 已经是叶子节点时, 解码出一个字符. 下次循环再次从编码树根节点出发, 解析下一个字符
                sentence = sentence+node.data
                node = pfc_tree.root
            
        return sentence

In [26]:
pfc_tree = PFCUtil.pfc_tree('abcdefghigklmnopqrstuvwxyz ')
code = PFCUtil.encode('i love you',pfc_tree)
print code
PFCUtil.decode(code,pfc_tree)

10010010010101110001111101110010000101100011110


'i love you'

#### 7. 最优编码树
1. 高效的编码算法, 生成的编码序列应该尽可能的短   
 使用`平均编码长度`度量编码质量. `平均编码长度`为所有字符编码长度的平均值. 该平均值越小越好  
2. 最优编码树  
 因为字符的编码长度 = 该字符叶子节点的深度, 所以`平均编码长度`最小的编码树, 就是`最优编码树`